In [ ]:
import requests
import pandas as pd
pd.options.display.max_columns=200
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import time

In [ ]:
import requests
import pandas as pd

def get_historical_data():
    # Prix et volume historiques
    url_market_chart = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': 'max',
        'interval': 'daily'
    }
    response = requests.get(url_market_chart, params=params)
    market_data = response.json()
    
    # Convertir les données en DataFrame
    df = pd.DataFrame(market_data['prices'], columns=['timestamp', 'price'])
    
    # Convertir le timestamp en datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    # Définir le timestamp comme index
    df.set_index('timestamp', inplace=True)
    
    return df

# Utilisation de la fonction
btc_data = get_historical_data()

# Afficher les premières lignes du DataFrame
btc_data

In [ ]:
def get_market_cap_data():
    # Récupérer la capitalisation boursière historique
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': 'max',
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    market_data = response.json()
    
    # Convertir les données en DataFrame
    df_market_cap = pd.DataFrame(market_data['market_caps'], columns=['timestamp', 'market_cap'])
    
    # Convertir le timestamp en datetime et le définir comme index
    df_market_cap['timestamp'] = pd.to_datetime(df_market_cap['timestamp'], unit='ms')
    df_market_cap.set_index('timestamp', inplace=True)
    
    return df_market_cap

# Utilisation de la fonction
market_cap_data = get_market_cap_data()

# Fusionner les données de capitalisation boursière avec btc_data
btc_data = btc_data.merge(market_cap_data, left_index=True, right_index=True, how='left')

# Afficher les premières lignes du DataFrame mis à jour
btc_data

In [ ]:
# Convertir l'index en dates et supprimer l'heure
btc_data.index = pd.to_datetime(btc_data.index).date

# Supprimer les doublons basés sur la date
btc_data = btc_data[~btc_data.index.duplicated(keep='first')]

In [ ]:
btc_data

In [ ]:
import time

# Votre clé API
api_key = input("Please enter your CryptoCompare API key: ")

# Convertit la date '28/04/2013' en timestamp UNIX
start_date = '12/10/2010 00:00:00'
start_timestamp = int(time.mktime(time.strptime(start_date, '%d/%m/%Y %H:%M:%S')))

# Timestamp pour aujourd'hui
end_timestamp = int(time.time())

# Limite de points de données par requête
limit = 2000

# Calcule le nombre total de jours et de requêtes nécessaires
total_days = (end_timestamp - start_timestamp) // (24 * 60 * 60)
total_requests = -(-total_days // limit)  # Utilise la division entière arrondie vers le haut

# Initialise un DataFrame vide
btc_data_full = pd.DataFrame()

# Fait plusieurs requêtes pour couvrir toute la période
for i in range(total_requests):
    # Ajuste le paramètre toTs pour chaque requête
    toTs = end_timestamp - i * limit * 24 * 60 * 60
    
    # Construit l'URL de l'API
    url = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym=BTC&tsym=USD&limit={limit}&toTs={toTs}&api_key={api_key}'
    
    # Fait la requête API
    response = requests.get(url)
    data = response.json()
    
    # Crée un DataFrame avec les données obtenues
    new_data = pd.DataFrame(data['Data']['Data'])
    new_data['time'] = pd.to_datetime(new_data['time'], unit='s')
    new_data.set_index('time', inplace=True)
    
    # Concaténe les nouveaux données avec les données existantes
    btc_data_full = pd.concat([new_data, btc_data_full])

# Trie le DataFrame combiné par index (date)
btc_data_full.sort_index(inplace=True)

# Filtre les données pour n'inclure que celles depuis le 28/04/2013
btc_data_full = btc_data_full[btc_data_full.index >= start_date]

# Affiche le DataFrame
btc_data_full

In [ ]:
btc_data_full.drop(['conversionSymbol', 'conversionType'], axis=1, inplace=True)

In [ ]:
duplicates = btc_data_full.index.duplicated(keep=False)
print(btc_data_full[duplicates])

In [ ]:
btc_data_full = btc_data_full[~btc_data_full.index.duplicated(keep='first')]

In [ ]:
btc_data

In [ ]:
btc_data_full['market_cap'] = btc_data['market_cap']

In [ ]:
btc_data = btc_data_full.copy()

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

btc_data["progression daily"] = 0.0
btc_data["progression tomorrow"] = 0.0

# Calcul de la progression quotidienne
btc_data.loc[:, "progression daily"] = btc_data['open'].pct_change()

# Calcul de la progression du lendemain
btc_data["progression tomorrow"] = btc_data["close"].values / btc_data["open"].values - 1

In [ ]:
btc_data

In [ ]:
# Affichage de l'histogramme de la progression quotidienne
plt.hist(btc_data['progression daily'].dropna(), bins=300, alpha=0.5)
plt.title('Répartition de la Progression Daily')
plt.xlabel('Progression Daily')
plt.ylabel('Fréquence')
plt.grid(True)
plt.show()

In [ ]:
print(btc_data['progression daily'].describe())

In [ ]:
btc_data.loc[btc_data['volumefrom']==0]

In [ ]:
btc_data['2011-06-15':'2011-06-30']

In [ ]:
# Calculez la médiane pour chaque colonne du 15 au 30 juin
median_values = btc_data['2011-06-15':'2011-06-30'].median()

# Affectez la médiane à chaque ligne de la période du 20 au 25 juin, colonne par colonne
for column in btc_data.columns:
    btc_data.loc['2011-06-20':'2011-06-25', column] = median_values[column]

In [ ]:
btc_data['2011-06-15':'2011-06-30']

In [ ]:
missing_values_sum = btc_data.isna().sum()

In [ ]:
missing_values_sum

In [ ]:
"""btc_data.loc[btc_data['market_cap'].isna()]"""

In [ ]:
"""missing_index = btc_data.loc[btc_data['market_cap'].isna()].index[0]

# Obtenir la position de l'index manquant
missing_pos = btc_data.index.get_loc(missing_index)

# Définir les positions de début et de fin pour l'extraction des données
start_pos = max(0, missing_pos - 3)
end_pos = min(len(btc_data), missing_pos + 4)

# Afficher les 3 lignes avant et après la valeur manquante
subset_btc_data = btc_data.iloc[start_pos:end_pos]
subset_btc_data"""

In [ ]:
"""# Trouver l'index de la valeur manquante
missing_index = btc_data.loc[btc_data['market_cap'].isna()].index[0]

# Obtenir la position de l'index manquant
missing_pos = btc_data.index.get_loc(missing_index)

# Calculer la moyenne des valeurs de 'market_cap' qui entourent la valeur manquante
mean_value = (btc_data['market_cap'].iloc[missing_pos - 1] + btc_data['market_cap'].iloc[missing_pos + 1]) / 2

# Remplacer la valeur manquante par la moyenne calculée
btc_data['market_cap'].iloc[missing_pos] = mean_value"""

In [ ]:
# Créer une variable cible binaire : 1 si la progression demain est positive, 0 sinon
btc_data['target'] = np.where(btc_data['progression tomorrow'] > 0, 1, 0)

In [ ]:
btc_data['target'].describe()

In [ ]:
btc_data.to_csv('data/btc_data')

In [ ]:
"""import pandas as pd
from pytrends.request import TrendReq
import matplotlib.pyplot as plt
import time
import random

# Configurer l'objet pytrends
pytrends = TrendReq()

kw_list = ["Bitcoin"]

# Liste des périodes
timeframes = ['2013-04-28 2018-04-27', '2018-04-28 2023-04-27']

temp = []

for timeframe in timeframes:
    pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo='', gprop='')
    temp.append(pytrends.interest_over_time())
    time.sleep(random.randint(10, 20))  # Ajoute une pause de X secondes entre chaque requête"""

In [ ]:
"""import pandas as pd
from pytrends.request import TrendReq
import matplotlib.pyplot as pltdatetime
from datetime import date

current_date = date.today()
formatted_date = current_date.strftime('%Y-%m-%d')
print(formatted_date)

# Configurer l'objet pytrends
pytrends = TrendReq(hl='en-US', tz=360)

# Définir la requête
kw_list = ["Bitcoin"]
pytrends.build_payload(kw_list, cat=0, timeframe='2023-04-28 ' + formatted_date, geo='', gprop='')

# Récupérer les données
df = pytrends.interest_over_time()
temp.append(df)

# Concaténer les dataframes
trends = pd.concat(temp, axis=0)

# Afficher les données
print(trends)


# Visualiser les données
plt.plot(df.index, df['Bitcoin'])
plt.title('Interest Over Time')
plt.xlabel('Date')
plt.ylabel('Trend Index')
plt.grid(True)
plt.show()"""

In [ ]:
btc_data.columns

In [ ]:
#Calcul EMA 26

# Choix du nombre de périodes pour l'EMA. Habituellement, 12, 26 ou 50 sont utilisés.
n = 26
alpha = 2 / (n + 1)

# Calcul de l'EMA
btc_data['ema_26'] = btc_data['open'].ewm(span=n, adjust=False).mean()

#Calcul EMA 12

# Choix du nombre de périodes pour l'EMA. Habituellement, 12, 26 ou 50 sont utilisés.
n = 12
alpha = 2 / (n + 1)

# Calcul de l'EMA
btc_data['ema_12'] = btc_data['open'].ewm(span=n, adjust=False).mean()

# Afficher le dataframe
btc_data.tail()

In [ ]:
# Calcul du MACD
btc_data['macd'] = btc_data['ema_12'] - btc_data['ema_26']

btc_data.tail()

In [ ]:
# Calcul du RSI 

# Calculer la différence de prix par rapport à la journée précédente
btc_data['delta'] = btc_data['open'].diff()

# Identifier les gains et les pertes
btc_data['gain'] = btc_data['delta'].where(btc_data['delta'] > 0, 0)
btc_data['loss'] = -btc_data['delta'].where(btc_data['delta'] < 0, 0)

# Calculer la moyenne des gains et des pertes sur 14 jours
rolling_window = 14
btc_data['avg_gain'] = btc_data['gain'].rolling(window=rolling_window).mean()
btc_data['avg_loss'] = btc_data['loss'].rolling(window=rolling_window).mean()

# Calculer le RS (Relative Strength)
btc_data['rs'] = btc_data['avg_gain'] / btc_data['avg_loss']

# Calculer le RSI
btc_data['rsi'] = 100 - (100 / (1 + btc_data['rs']))

# Supprimer les colonnes intermédiaires
btc_data.drop(columns=['delta', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs'], inplace=True)

btc_data.tail()

In [ ]:
# Calcul du volume relatif

# Période pour calculer la moyenne du volume
rolling_window = 14

# Calculer la moyenne du volume sur la période donnée
btc_data['avg_volume'] = btc_data['volumeto'].rolling(window=rolling_window).mean()

# Calculer le volume relatif
btc_data['relative_volume'] = btc_data['volumeto'] / btc_data['avg_volume']

# Supprimer la colonne de volume moyen intermédiaire si désiré
btc_data.drop(columns=['avg_volume'], inplace=True)

btc_data.tail()


In [ ]:
# Calcul de l'OBV (On-Balance Volume)

# Calculer la direction du mouvement des prix
btc_data['price_direction'] = btc_data['open'].diff()

# Calculer l'OBV
btc_data['obv'] = btc_data['volumeto'].where(btc_data['price_direction'] > 0, -btc_data['volumeto']).cumsum()

# Supprimer la colonne intermédiaire de direction des prix
btc_data.drop(columns=['price_direction'], inplace=True)

btc_data.tail()


In [ ]:
# Calcul de l'ATR (Average True Range)

# Calculer la différence de prix de clôture par rapport à la journée précédente
btc_data['prev_close'] = btc_data['open'].shift(1)

# Calculer les trois composantes du True Range
btc_data['high_minus_low'] = btc_data['high'] - btc_data['low']
btc_data['high_minus_prev_close'] = abs(btc_data['high'] - btc_data['prev_close'])
btc_data['low_minus_prev_close'] = abs(btc_data['low'] - btc_data['prev_close'])

# Déterminer le True Range comme étant le maximum des trois valeurs précédentes
btc_data['tr'] = btc_data[['high_minus_low', 'high_minus_prev_close', 'low_minus_prev_close']].max(axis=1)

# Calculer l'ATR comme étant la moyenne mobile du TR sur une période de 14 jours
rolling_window = 14
btc_data['atr'] = btc_data['tr'].rolling(window=rolling_window).mean()

# Supprimer les colonnes intermédiaires
columns_to_drop = ['prev_close', 'high_minus_low', 'high_minus_prev_close', 'low_minus_prev_close', 'tr']
btc_data.drop(columns=columns_to_drop, inplace=True)

btc_data.tail()

In [ ]:
# Calcul des bandes de Bollinger

rolling_window = 20

# Moyenne mobile simple
btc_data['sma'] = btc_data['open'].rolling(window=rolling_window).mean()

# Écart-type des prix sur la période
btc_data['price_std'] = btc_data['open'].rolling(window=rolling_window).std()

# Calcul des bandes de Bollinger
btc_data['bollinger_upper'] = btc_data['sma'] + (btc_data['price_std'] * 2)
btc_data['bollinger_lower'] = btc_data['sma'] - (btc_data['price_std'] * 2)

# Suppression des colonnes intermédiaires
btc_data.drop(columns=['sma', 'price_std'], inplace=True)

btc_data.tail()

In [ ]:
# Oscillateur Stochastique

rolling_window = 14

# Trouver le prix le plus bas et le plus élevé sur la période
btc_data['rolling_low'] = btc_data['low'].rolling(window=rolling_window).min()
btc_data['rolling_high'] = btc_data['high'].rolling(window=rolling_window).max()

# Calcul du Stochastic Oscillator
btc_data['k'] = 100 * ((btc_data['open'] - btc_data['rolling_low']) / (btc_data['rolling_high'] - btc_data['rolling_low']))

# Suppression des colonnes intermédiaires
btc_data.drop(columns=['rolling_low', 'rolling_high'], inplace=True)

btc_data.tail()

In [ ]:
# Momentum

n_days = 10

# Calcul du Momentum
btc_data['momentum'] = btc_data['open'] - btc_data['open'].shift(n_days)

btc_data.tail()

In [ ]:
missing_values_sum = btc_data.isna().sum()
missing_values_sum

In [ ]:
url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=max&interval=daily"
response = requests.get(url)
data = response.json()

In [ ]:
#Fear and greed index

# URL de base
base_url = "https://api.alternative.me"

# Endpoint pour le Fear and Greed Index
endpoint = "/fng/"

# Paramètres
params = {
    "limit": "0",  # obtenir toutes les données disponibles
    "format": "json",  # format des données
    "date_format": "world"  # format de la date
}

# Faire la requête
response = requests.get(base_url + endpoint, params=params)

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()["data"]
    df_fng = pd.DataFrame(data)
else:
    print(f"Erreur {response.status_code}: {response.text}")

In [ ]:
df_fng.drop(df_fng.columns[-1], axis=1, inplace=True)

In [ ]:
df_fng

In [ ]:
'''import requests
import json

# URL de l'endpoint GraphQL de Bitquery
url = "https://graphql.bitquery.io/"

api_key = input("Enter yout bitquery API key :")

# Headers de la requête
headers = {
    "Content-Type": "application/json",
    "X-API-KEY": api_key
}

# Votre requête GraphQL
query = """
{
  bitcoin(network: bitcoin) {
    blocks(time: {since: "2013-04-28T00:00:00Z", till: "2013-05-01T00:00:00Z"}) {
      timestamp {
        time(format: "%Y-%m-%d %H:%M:%S")
      }
      difficulty
    }
  }
}
"""



# Effectuer la requête
response = requests.post(url, headers=headers, data=json.dumps({"query": query}))

# Vérifier si la requête a réussi
if response.status_code == 200:
    # Afficher les données retournées par l'API
    print(json.dumps(response.json(), indent=2))
else:
    # Afficher l'erreur
    print("Erreur :", response.status_code)
    print(response.text)'''

In [ ]:
'''# Dates de début et de fin
start_date = datetime.strptime("2013-04-28", "%Y-%m-%d")
end_date = datetime.now()

# Créer un DataFrame pour stocker les données
data = []

# Boucle sur chaque jour entre start_date et end_date
current_date = start_date
while current_date <= end_date:
    # Convertir les dates en chaînes de caractères au format ISO 8601
    since = current_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    till = (current_date + timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
    
    # Votre requête GraphQL
    query = f"""
    {{
      bitcoin(network: bitcoin) {{
        blocks(time: {{since: "{since}", till: "{till}"}}) {{
          timestamp {{
            time(format: "%Y-%m-%d %H:%M:%S")
          }}
          difficulty
        }}
      }}
    }}
    """"""
    
    # Effectuer la requête
    response = requests.post(url, headers=headers, data=json.dumps({"query": query}))
    
    # Vérifier si la requête a réussi
    if response.status_code == 200:
      # Extraire les données
      blocks = response.json()['data']['bitcoin']['blocks']
      if blocks:
          # Ajouter les données au DataFrame
          for block in blocks:
              data.append([block['timestamp']['time'], block['difficulty']])
    elif response.status_code == 429:
        # Attendre et réessayer
        print("Trop de requêtes, en attente...")
        time.sleep(10)  # Attendre 10 secondes avant de réessayer
        continue  # Réessayer la même requête
    else:
        # Afficher l'erreur
        print("Erreur :", response.status_code)
        print(response.text)
    
    # Passer au jour suivant
    current_date += timedelta(days=1)
    
    # Attendre avant la prochaine requête pour éviter de dépasser la limite de taux
    time.sleep(1)  # Attendre 1 seconde avant la prochaine requête

# Convertir les données en DataFrame
dif = pd.DataFrame(data, columns=['timestamp', 'difficulty'])

# Afficher le DataFrame
print(dif)'''

In [ ]:
from datetime import datetime, timedelta
import time
import requests
import json
import pandas as pd

'''# Dates de début et de fin
start_date = datetime.strptime("2013-04-28", "%Y-%m-%d")
end_date = datetime.now()

# Créer un DataFrame pour stocker les données
data = []
nb = 0

# Boucle sur chaque jour entre start_date et end_date
current_date = start_date
while current_date <= end_date:
    # Convertir les dates en chaînes de caractères au format ISO 8601
    since = current_date.strftime("%Y-%m-%dT00:00:00Z")  # Minuit du jour courant
    till = (current_date + timedelta(hours=2)).strftime("%Y-%m-%dT02:00:00Z")  # 02 du jour courant
    
    # Votre requête GraphQL
    query = f"""
    {{
      bitcoin(network: bitcoin) {{
        blocks(time: {{since: "{since}", till: "{till}"}}) {{
          timestamp {{
            time(format: "%Y-%m-%d %H:%M:%S")
          }}
          difficulty
        }}
      }}
    }}
    """
    
    # Effectuer la requête
    response = requests.post(url, headers=headers, data=json.dumps({"query": query}))
    
    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Extraire les données
        blocks = response.json()['data']['bitcoin']['blocks']
        if blocks:
            # Ajouter les données au DataFrame
            block = blocks[0]
            data.append([block['timestamp']['time'], block['difficulty']])
    elif response.status_code == 429:
        # Attendre et réessayer
        print("Trop de requêtes, en attente...")
        time.sleep(10)  # Attendre 10 secondes avant de réessayer
        continue  # Réessayer la même requête
    else:
        # Afficher l'erreur
        print("Erreur :", response.status_code)
        print(response.text)
    
    # Passer au jour suivant
    current_date += timedelta(days=1)
    nb += 1
    print(nb)
    print(data)
    
    # Attendre avant la prochaine requête pour éviter de dépasser la limite de taux
    time.sleep(1)  # Attendre 1 seconde avant la prochaine requête

# Convertir les données en DataFrame
dif = pd.DataFrame(data, columns=['timestamp', 'difficulty'])

# Afficher le DataFrame
print(dif)'''

In [ ]:
"""
dif['timestamp'] = pd.to_datetime(dif['timestamp'])

dif['timestamp'] = dif['timestamp'].dt.strftime('%Y-%m-%d')

dif.to_csv('data/dif.csv')"""

In [ ]:
dif = pd.read_csv('data/dif.csv', index_col=0)
dif

In [ ]:
import requests
import json

# URL de l'endpoint GraphQL de Bitquery
url = "https://graphql.bitquery.io/"

api_key = input("Enter yout bitquery API key :")

# Headers de la requête
headers = {
    "Content-Type": "application/json",
    "X-API-KEY": api_key
}

# Dates de début et de fin
start_date = datetime.strptime("2023-10-13", "%Y-%m-%d")
end_date = datetime.now()

# Créer une liste pour stocker les nouvelles données
new_data = []

# Boucle sur chaque jour entre start_date et end_date
current_date = start_date
while current_date <= end_date:
    # Convertir les dates en chaînes de caractères au format ISO 8601
    since = current_date.strftime("%Y-%m-%dT00:00:00Z")  # Minuit du jour courant
    till = (current_date + timedelta(hours=2)).strftime("%Y-%m-%dT02:00:00Z")  # 02 du jour courant
        

      # Votre requête GraphQL
    query = f"""
    {{
      bitcoin(network: bitcoin) {{
        blocks(time: {{since: "{since}", till: "{till}"}}) {{
          timestamp {{
            time(format: "%Y-%m-%d")
          }}
          difficulty
        }}
      }}
    }}
    """
    
    # Effectuer la requête
    response = requests.post(url, headers=headers, data=json.dumps({"query": query}))
    
    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Extraire les données
        blocks = response.json()['data']['bitcoin']['blocks']
        if blocks:
            # Ajouter les données à la liste new_data
            block = blocks[0]
            new_data.append([block['timestamp']['time'], block['difficulty']])
    elif response.status_code == 429:
        # Attendre et réessayer
        print("Trop de requêtes, en attente...")
        time.sleep(10)  # Attendre 10 secondes avant de réessayer
        continue  # Réessayer la même requête
    else:
        # Afficher l'erreur
        print("Erreur :", response.status_code)
        print(response.text)
    
    # Passer au jour suivant
    current_date += timedelta(days=1)
    
    # Attendre avant la prochaine requête pour éviter de dépasser la limite de taux
    time.sleep(1)  # Attendre 1 seconde avant la prochaine requête

# Convertir les nouvelles données en DataFrame
new_dif = pd.DataFrame(new_data, columns=['timestamp', 'difficulty'])

# Ajouter les nouvelles données au dataframe original `dif`
difficulty = pd.concat([dif, new_dif], ignore_index=True)

# Afficher le DataFrame mis à jour
difficulty

In [ ]:
difficulty.set_index('timestamp', inplace=True)

In [ ]:
btc_data.index = pd.to_datetime(btc_data.index)
difficulty.index = pd.to_datetime(difficulty.index)

In [ ]:
# Ajouter difficulty au DataFrame final
btc_data = btc_data.merge(difficulty, left_index=True, right_index=True, how='outer')

In [ ]:
btc_data

In [ ]:
btc_data.drop(btc_data.index[:22], inplace=True)
btc_data

In [ ]:
missing_values_sum = btc_data.isna().sum()
missing_values_sum

In [ ]:
btc_data.loc[(btc_data['market_cap'].isna()) & (~btc_data['difficulty'].isna())]

In [ ]:
# Trouver les dates qui satisfont les conditions
dates = btc_data[(btc_data['market_cap'].isna()) & (~btc_data['difficulty'].isna())].index

# Pour chaque date, afficher la ligne précédente, la ligne actuelle et la ligne suivante
for date in dates:
    # Obtenez la date précédente et la date suivante
    prev_date = date - pd.DateOffset(days=1)
    next_date = date + pd.DateOffset(days=1)
    
    # Affichez les lignes correspondantes
    print(btc_data.loc[prev_date:next_date, 'market_cap'])
    print("\n" + "="*50 + "\n")  # Séparateur pour une meilleure lisibilité

In [ ]:
# Trouver les dates qui satisfont les conditions
dates_with_na = btc_data[(btc_data['market_cap'].isna()) & (~btc_data['difficulty'].isna())].index

# Pour chaque date avec une valeur manquante, remplacez-la par la moyenne des deux valeurs qui l'entourent
for date in dates_with_na:
    # Obtenez la date précédente et la date suivante
    prev_date = date - pd.DateOffset(days=1)
    next_date = date + pd.DateOffset(days=1)
    
    # Calculez la moyenne des deux valeurs
    average_value = (btc_data.loc[prev_date, 'market_cap'] + btc_data.loc[next_date, 'market_cap']) / 2
    
    # Remplacez la valeur manquante par la moyenne calculée
    btc_data.loc[date, 'market_cap'] = average_value

In [ ]:
btc_data.loc[(btc_data['market_cap'].isna()) & (~btc_data['difficulty'].isna())]

In [ ]:
#Save
btc_data.to_csv('data/btc_data.csv')